In [59]:
import eigen as e
import rbdyn as rbd
import sva as s
from rbdyn.parsers import *

import numpy as np

urdf_file = "./assets/kuka_iiwa.urdf"

robot = from_urdf_file(urdf_file)
robot.mbc.gravity = e.Vector3d(0, 0, 9.81)
robot.mbc.zero(robot.mb)

# q = np.array([1,2,4,4,5,6,3])
q = np.array([1,2,4,4,2,3,1])
dq = -np.array([1,2,3,4,5,6,7])

robot.mbc.q = [
    [],
    [q[0]],
    [q[1]],
    [q[2]],
    [q[3]],
    [q[4]],
    [q[5]],
    [q[6]],
]
robot.mbc.alpha = [
    [],
    [dq[0]],
    [dq[1]],
    [dq[2]],
    [dq[3]],
    [dq[4]],
    [dq[5]],
    [dq[6]],
]

sel_mat = e.MatrixXd.Zero(6, 6)
sel_mat.coeff(0, 3, 1)
sel_mat.coeff(1, 4, 1)
sel_mat.coeff(2, 5, 1)
sel_mat.coeff(3, 0, 1)
sel_mat.coeff(4, 1, 1)
sel_mat.coeff(5, 2, 1)
        
rbd.forwardKinematics(robot.mb, robot.mbc)
rbd.forwardVelocity(robot.mb, robot.mbc)

fd = rbd.ForwardDynamics(robot.mb)
fd.computeH(robot.mb, robot.mbc)
M = fd.H()
Minv = M.inverse()
fd.computeC(robot.mb, robot.mbc)
h = fd.C()

jac = rbd.Jacobian(robot.mb, robot.mb.bodies()[-1].name())
swapped_jac = e.MatrixXd(6, robot.mb.nrDof())
jac.fullJacobian(robot.mb, jac.jacobian(robot.mb, robot.mbc), swapped_jac)
J = sel_mat * swapped_jac
swapped_jac_dot = e.MatrixXd(6, robot.mb.nrDof()) 
jac.fullJacobian(robot.mb, jac.jacobianDot(robot.mb, robot.mbc), swapped_jac_dot)
dJ = sel_mat * swapped_jac_dot

Lambda_inv = J*Minv*J.transpose()
# mat_np = np.array(Lambda_inv)
# reg = 0 * np.identity(np.size(mat_np,0))
Lambda = Lambda_inv.inverse()


In [61]:
print("Lambda is \n{}".format(Lambda))

print("M is \n{}".format(M))

print("J^T is \n{}".format(J.transpose()))

print("dJ is \n{}".format(dJ))

Jbar = Minv*J.transpose()*Lambda
N1 = e.MatrixXd.Identity(7,7) - J.transpose() * Jbar.transpose()

J2 = e.MatrixXd.Zero(1, 7)
J2.coeff(0, 3, 1)
J21 = J2*N1 
Lambda21_inv = J21*Minv*J21.transpose()
Lambda21 = Lambda21_inv.inverse()

P = J.transpose()*Lambda*J
Q = J.transpose()*Lambda*J + J21.transpose()*Lambda21*J21

# print(J)
# print(J21)

# J_ = np.stack((np.array(J),np.array(J21)))

# print("P is \n{}".format(P.inverse()))
# print("Q is \n{}".format(Q.inverse()))

Lambda is 
  4.86688 -0.444278   1.86201  0.396878 -0.248812 -0.263834
-0.444278   10.3131  -5.10794  0.876909 -0.533796 -0.571399
  1.86201  -5.10794   8.63353 -0.195679  0.123588  0.130742
 0.396878  0.876909 -0.195679   0.23421 -0.139543 -0.150212
-0.248812 -0.533796  0.123588 -0.139543 0.0899938 0.0938575
-0.263834 -0.571399  0.130742 -0.150212 0.0938575   0.10063
M is 
     1.18382    0.0573391     -0.25467    0.0693474  1.50451e-05   0.00342671 -0.000798583
   0.0573391      1.15989     0.157948    0.0693697 -0.000180274  -0.00183145  0.000454092
    -0.25467     0.157948     0.309852  0.000620943 -0.000628145   0.00207587  0.000691547
   0.0693474    0.0693697  0.000620943     0.513196   0.00388039   0.00125557  -0.00012832
 1.50451e-05 -0.000180274 -0.000628145   0.00388039    0.0103803  4.27677e-07 -0.000989992
  0.00342671  -0.00183145   0.00207587   0.00125557  4.27677e-07   0.00582065 -4.89659e-15
-0.000798583  0.000454092  0.000691547  -0.00012832 -0.000989992 -4.89659e-15

In [42]:
import numpy as np
import eigen as e

a = np.random.rand(3,3)
b = np.random.rand(3,3)
c = np.vstack((a,b))

print(e.MatrixXd(a))
print(e.MatrixXd(b))
print(e.MatrixXd(c))

 0.636896  0.655267 0.0438742
  0.72304  0.914985   0.45259
0.0460359  0.532458  0.945114
 0.866162  0.864354  0.473918
 0.673476 0.0353093  0.496175
 0.454896  0.368581  0.816526
 0.636896  0.655267 0.0438742
  0.72304  0.914985   0.45259
0.0460359  0.532458  0.945114
 0.866162  0.864354  0.473918
 0.673476 0.0353093  0.496175
 0.454896  0.368581  0.816526


In [24]:
%matplotlib widget
import ipywidgets as widgets
import os
import matplotlib.pyplot as plt
from pytransform3d.urdf import UrdfTransformManager
import math

plt.rcParams['figure.dpi'] = 150

BASE_DIR = "assets/"
data_dir = BASE_DIR
search_path = "."
while (not os.path.exists(data_dir) and
       os.path.dirname(search_path) != "riemannian_mot_pol"):
    search_path = os.path.join(search_path, "..")
    data_dir = os.path.join(search_path, BASE_DIR)

tm = UrdfTransformManager()
with open(os.path.join(data_dir, "kuka_iiwa.urdf"), "r") as f:
    tm.load_urdf(f.read(), mesh_path=data_dir)

ax = tm.plot_frames_in(
    "lbr_iiwa_link_0", s=0.2, whitelist=["lbr_iiwa_link_0", "lbr_iiwa_link_1", "lbr_iiwa_link_7"],
    show_name=True) # plot all frames in a given reference frame
ax = tm.plot_connections_in("lbr_iiwa_link_0", ax=ax) # plot connections between parent and child frames
tm.plot_visuals("lbr_iiwa_link_0", ax=ax, wireframe=False, convex_hull_of_mesh=False) # plot all visuals in a given reference frame

ax.set_xlim((-1, 1))
ax.set_ylim((-1, 1))
ax.set_zlim((0.0, 1))
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
%matplotlib widget
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from pytransform3d.transformations import plot_transform
from pytransform3d.plot_utils import plot_sphere, remove_frame


random_state = np.random.RandomState(42)
ax = plot_sphere(
    radius=0.5, wireframe=False, alpha=0.1, color="k", n_steps=20, ax_s=1)
plot_sphere(ax=ax, radius=0.5, wireframe=True)
plot_transform(ax=ax, A2B=np.eye(4), s=0.3, lw=3)
remove_frame(ax)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
%matplotlib widget
import matplotlib.pyplot as plt
from pytransform3d.transformations import plot_transform
from pytransform3d.plot_utils import make_3d_axis

ax = make_3d_axis(ax_s=1, unit="m", n_ticks=6)
plot_transform(ax=ax)
plt.tight_layout()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …